In [ ]:
# Importamos las bibliotecas necesarias
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import LabelEncoder

# Creamos un conjunto de datos de ejemplo
# Cada fila representa una compra, con un ID de cliente y la herramienta comprada
data = {
    'cliente_id': [1, 1, 2, 2, 2, 3, 3, 3, 4, 4],
    'herramienta': ['sierra', 'martillo', 'sierra', 'taladro', 'tornillo', 'martillo', 'taladro', 'clavo', 'sierra', 'tornillo']
}

# Convertimos el conjunto de datos en un DataFrame de pandas
compras_df = pd.DataFrame(data)
print("Datos de compras:")
print(compras_df)

# Codificamos las herramientas a valores numéricos para facilitar el procesamiento
le = LabelEncoder()
compras_df['herramienta_codificada'] = le.fit_transform(compras_df['herramienta'])
print("\nDatos de compras con herramientas codificadas:")
print(compras_df)

# Creamos una matriz que relaciona clientes con las herramientas que han comprado
# Las filas son clientes y las columnas son herramientas codificadas
matriz_compras = compras_df.pivot_table(index='cliente_id', columns='herramienta_codificada', aggfunc='size', fill_value=0)
print("\nMatriz de compras cliente-herramienta:")
print(matriz_compras)

# Calculamos la similitud del coseno entre los clientes
# Esto nos permite ver qué tan similares son los patrones de compra de los clientes
similitud_clientes = cosine_similarity(matriz_compras)
print("\nSimilitud entre clientes:")
print(pd.DataFrame(similitud_clientes, index=matriz_compras.index, columns=matriz_compras.index))

# Definimos una función para recomendar herramientas
def recomendar_herramientas(cliente_id, matriz_compras, similitud_clientes, compras_df):
    # Obtenemos el índice del cliente en la matriz
    cliente_idx = matriz_compras.index.get_loc(cliente_id)

    # Extraemos la similitud de este cliente con los demás
    similitudes = similitud_clientes[cliente_idx]

    # Ordenamos los clientes según la similitud en orden descendente
    clientes_similares = matriz_compras.index[similitudes.argsort()[::-1]]

    # Recolectamos las herramientas compradas por clientes similares
    herramientas_recomendadas = []
    for cliente_similar in clientes_similares:
        if cliente_similar == cliente_id:
            continue
        herramientas_cliente = compras_df[compras_df['cliente_id'] == cliente_similar]['herramienta'].tolist()
        herramientas_recomendadas.extend(herramientas_cliente)

    # Eliminamos las herramientas que el cliente ya compró
    herramientas_compradas = compras_df[compras_df['cliente_id'] == cliente_id]['herramienta'].tolist()
    herramientas_recomendadas = list(set(herramientas_recomendadas) - set(herramientas_compradas))

    # Devolvemos las primeras 3 recomendaciones para simplicidad
    return herramientas_recomendadas[:3]

# Probamos la función de recomendación con un cliente de ejemplo
cliente_id = 1
recomendaciones = recomendar_herramientas(cliente_id, matriz_compras, similitud_clientes, compras_df)
print(f"\nHerramientas recomendadas para el cliente {cliente_id}: {recomendaciones}")


Datos de compras:
   cliente_id herramienta
0           1      sierra
1           1    martillo
2           2      sierra
3           2     taladro
4           2    tornillo
5           3    martillo
6           3     taladro
7           3       clavo
8           4      sierra
9           4    tornillo

Datos de compras con herramientas codificadas:
   cliente_id herramienta  herramienta_codificada
0           1      sierra                       2
1           1    martillo                       1
2           2      sierra                       2
3           2     taladro                       3
4           2    tornillo                       4
5           3    martillo                       1
6           3     taladro                       3
7           3       clavo                       0
8           4      sierra                       2
9           4    tornillo                       4

Matriz de compras cliente-herramienta:
herramienta_codificada  0  1  2  3  4
cliente_id          